<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250621%5D%20CM_%20FedAvg%20MLP-LSTM_logged_extended%20on%20WSN-BFSF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***20250605 FedAvg MLP-LSTM_logged_extended on WSN-BFSF***

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, InputLayer
from keras.utils import to_categorical
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
# --- Imports ---------------------------------------------------------------
import numpy as np, time, tensorflow as tf

In [ ]:
!pip install imbalanced-learn

In [ ]:
# ----------------------------
# Preprocessing and Partitioning
# ----------------------------
df = pd.read_csv("dataset.csv").dropna()
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = LabelEncoder().fit_transform(df[col])

X, y = df.drop('Class', axis=1), df['Class']
X = StandardScaler().fit_transform(X)
X, y = SMOTE(random_state=42).fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

In [ ]:
# --- Federated parameters ---------------------------------------------------
NUM_CLIENTS = 5
ROUNDS       = 75
LOCAL_EPOCHS = 1
BATCH_SIZE   = 32

# --- Data -------------------------------------------------------------------
# X_train: (N, T, 1)  | y_train: (N,)
# Make sure X_train is already 3-D!
client_X = np.array_split(X_train, NUM_CLIENTS, axis=0)
client_y = np.array_split(y_train, NUM_CLIENTS, axis=0)

X_test   = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
y_test_ohe = to_categorical(y_test)
NUM_CLASSES = y_test_ohe.shape[1]
INPUT_SHAPE = (X_test.shape[1], 1)

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
# --- Model builder ----------------------------------------------------------
def build_model(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES,
                lstm_units=64, dense_units=(128, 64), drop=0.30):
    model = Sequential([InputLayer(input_shape=input_shape),
                        LSTM(lstm_units, activation='tanh')])
    for u in dense_units:
        model.add(Dense(u, activation='relu'))
        model.add(Dropout(drop))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# --- Resource profile (lightweight) ----------------------------------------
global_model = build_model()
tot_params   = global_model.count_params()
model_sizeMB = round((tot_params * 4) / 2**20, 2)      # 4 bytes / param
COMM_MB_UP   = model_sizeMB * NUM_CLIENTS              # uplink only

print(f"[Profile] Params={tot_params:,}  Size={model_sizeMB} MB")

# --- Federated loop ---------------------------------------------------------
global_weights = global_model.get_weights()
history = []

for r in range(1, ROUNDS + 1):
    t0 = time.time()
    client_weights, client_sizes = [], []

    # -------- local training --------
    for Xi, yi in zip(client_X, client_y):
        local = build_model()
        local.set_weights(global_weights)
        local.fit(Xi, to_categorical(yi, NUM_CLASSES),
                  epochs=LOCAL_EPOCHS,
                  batch_size=BATCH_SIZE,
                  verbose=0)
        client_weights.append(local.get_weights())
        client_sizes.append(len(yi))

    # -------- FedAvg aggregation ----
    total_samples = sum(client_sizes)
    new_weights = []
    for layer in range(len(global_weights)):
        layer_updates = np.array([
            (n / total_samples) * w[layer]
            for w, n in zip(client_weights, client_sizes)
        ])
        new_weights.append(np.sum(layer_updates, axis=0))
    global_weights = new_weights
    global_model.set_weights(global_weights)

    # -------- Evaluation ------------
    y_prob = global_model.predict(X_test, verbose=0)
    y_pred = np.argmax(y_prob, axis=1)
    y_true = np.argmax(y_test_ohe, axis=1)

    metrics = dict(
        Round=r,
        Accuracy=accuracy_score(y_true, y_pred),
        Precision=precision_score(y_true, y_pred, average='weighted'),
        Recall=recall_score(y_true, y_pred, average='weighted'),
        F1=f1_score(y_true, y_pred, average='weighted'),
        AUC=roc_auc_score(y_test_ohe, y_prob, multi_class='ovr',
                          average='weighted'),
        Time_s=round(time.time() - t0, 2),
        Comm_MB=COMM_MB_UP           # add +downlink if desired
    )
    history.append(metrics)
    print(f"Round {r:02d}: acc={metrics['Accuracy']:.8f} "
          f"F1={metrics['F1']:.8f}  comm={metrics['Comm_MB']} MB")

# history is a list of dicts ready for a Pandas DataFrame

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


[Profile] Params=33,732  Size=0.13 MB
Round 01: acc=0.75864201 F1=0.75097365  comm=0.65 MB


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Round 02: acc=0.89586791 F1=0.89338745  comm=0.65 MB


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

Round 73: acc=0.98659413 F1=0.98658750  comm=0.65 MB


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Round 74: acc=0.98674155 F1=0.98673959  comm=0.65 MB


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Round 75: acc=0.98633733 F1=0.98633727  comm=0.65 MB


In [ ]:
# ----------------------------
# Save Log
# ----------------------------
# Use the correctly named variable 'history' to create the DataFrame
history_df = pd.DataFrame(history)
# Correct the filename to match the download call
log_filename = "fedavg_training_and_resource_log_WSN-BFSF.csv"
# Use the correctly named DataFrame 'history_df' to save to CSV
history_df.to_csv(log_filename, index=False)
print(f"\nLog saved to {log_filename}")


# ----------------------------
# Download Log
# ----------------------------
from google.colab import files
# Use the corrected filename for download
files.download(log_filename)


Log saved to fedavg_training_and_resource_log_WSN-BFSF.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# prompt: Please display all the resource that used in this training model

print("Total parameters:", tot_params)
print("Model size:", model_sizeMB, "MB")
print("Communication per round (uplink):", COMM_MB_UP, "MB")
print("\nDetailed resource usage per round:")
print(history_df[['Round', 'Time_s', 'Comm_MB']])


Total parameters: 33732
Model size: 0.13 MB
Communication per round (uplink): 0.65 MB

Detailed resource usage per round:
    Round  Time_s  Comm_MB
0       1  216.56     0.65
1       2  206.94     0.65
2       3  199.82     0.65
3       4  194.71     0.65
4       5  195.98     0.65
..    ...     ...      ...
70     71  173.14     0.65
71     72  170.69     0.65
72     73  168.35     0.65
73     74  166.21     0.65
74     75  165.63     0.65

[75 rows x 3 columns]


In [ ]:
# ---------------------------------------------------------
#  Confusion-matrix block  –  paste just before the script
#  finishes (after the last evaluation / logging section).
# ---------------------------------------------------------
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# --- 1.  Re-evaluate the final model ---------------------
y_prob_final = global_model.predict(X_test, verbose=0)   # FedAvg script
# y_prob_final = g_model.predict(X_te, verbose=0)         # TrustFed script
y_pred_final = np.argmax(y_prob_final, axis=1)
y_true_final = y_test                                    # already integers

# --- 2.  Confusion matrix (raw counts) ------------------
cm = confusion_matrix(y_true_final, y_pred_final)

# Optional: label list; replace with real class names if you have them
class_labels = [f"C{c}" for c in range(cm.shape[0])]

# --- 3.  Save the raw matrix for archival ---------------
# Define DATA_PATH as the filename of the dataset
DATA_PATH = "dataset.csv"
# Construct the cm_path using the current directory
cm_path = f"cm_{DATA_PATH.split('/')[-1].split('.')[0]}_FedAvg.csv" # Removed '/mnt/data/'
np.savetxt(cm_path, cm, delimiter=",", fmt="%d")
print("Confusion-matrix CSV written to", cm_path)

# --- 4.  Make a heat-map figure -------------------------
plt.figure(figsize=(4.5,4))
sns.heatmap(cm,
            annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=class_labels, yticklabels=class_labels,
            linewidths=.5, linecolor='grey')
plt.xlabel("Predicted label")
plt.ylabel("True label")
# Use a fixed string "FedAvg" for the title
plt.title("Confusion Matrix – FedAvg-IDS")
plt.tight_layout()

fig_path = cm_path.replace(".csv", ".png")
plt.savefig(fig_path, dpi=300)        # -> e.g. cm_WSN-BFSF_FedAvg.png
plt.close()
print("Figure saved to", fig_path)
# ---------------------------------------------------------

Confusion-matrix CSV written to cm_dataset_FedAvg.csv
Figure saved to cm_dataset_FedAvg.png


In [ ]:
# prompt: download the Confusion-matrix CSV and Figure

# Correct the filenames to match the saving process
cm_path = "confusion_matrix_FedAvg_BFSF.csv"
fig_path = "confusion_matrix_FedAvg_BFSF.png"

# Save the raw matrix for archival
np.savetxt(cm_path, cm, delimiter=",", fmt="%d")
print("Confusion-matrix CSV written to", cm_path)

# Make a heat-map figure
plt.figure(figsize=(4.5,4))
sns.heatmap(cm,
            annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=class_labels, yticklabels=class_labels,
            linewidths=.5, linecolor='grey')
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Confusion Matrix – FedAvg")
plt.tight_layout()

plt.savefig(fig_path, dpi=300)
plt.close()
print("Figure saved to", fig_path)

# Download the Confusion-matrix CSV and Figure
files.download(cm_path)
files.download(fig_path)

Confusion-matrix CSV written to confusion_matrix_FedAvg_BFSF.csv
Figure saved to confusion_matrix_FedAvg_BFSF.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>